# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbea0bdff10>
├── 'a' --> tensor([[ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263]])
└── 'x' --> <FastTreeValue 0x7fbea0bdffa0>
    └── 'c' --> tensor([[-0.4409,  0.5967,  1.4386, -1.2170],
                        [-0.7715,  0.1039,  0.0344,  0.4698],
                        [-1.0914,  1.5996,  1.2745, -1.0413]])

In [4]:
t.a

tensor([[ 0.5674, -0.5892,  0.1715],
        [-0.0126,  0.8789,  1.1263]])

In [5]:
%timeit t.a

64.9 ns ± 0.0382 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbea0bdff10>
├── 'a' --> tensor([[ 0.5644,  2.1224,  0.7770],
│                   [ 0.3824, -0.6941,  1.4970]])
└── 'x' --> <FastTreeValue 0x7fbea0bdffa0>
    └── 'c' --> tensor([[-0.4409,  0.5967,  1.4386, -1.2170],
                        [-0.7715,  0.1039,  0.0344,  0.4698],
                        [-1.0914,  1.5996,  1.2745, -1.0413]])

In [7]:
%timeit t.a = new_value

60.5 ns ± 0.0459 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263]]),
    x: Batch(
           c: tensor([[-0.4409,  0.5967,  1.4386, -1.2170],
                      [-0.7715,  0.1039,  0.0344,  0.4698],
                      [-1.0914,  1.5996,  1.2745, -1.0413]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5674, -0.5892,  0.1715],
        [-0.0126,  0.8789,  1.1263]])

In [11]:
%timeit b.a

53.8 ns ± 0.0434 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8132,  0.4155, -1.5721],
               [-1.4470,  0.5987,  1.4111]]),
    x: Batch(
           c: tensor([[-0.4409,  0.5967,  1.4386, -1.2170],
                      [-0.7715,  0.1039,  0.0344,  0.4698],
                      [-1.0914,  1.5996,  1.2745, -1.0413]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.0725 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 2.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 25.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 456 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbdf2b39fa0>
├── 'a' --> tensor([[[ 0.5674, -0.5892,  0.1715],
│                    [-0.0126,  0.8789,  1.1263]],
│           
│                   [[ 0.5674, -0.5892,  0.1715],
│                    [-0.0126,  0.8789,  1.1263]],
│           
│                   [[ 0.5674, -0.5892,  0.1715],
│                    [-0.0126,  0.8789,  1.1263]],
│           
│                   [[ 0.5674, -0.5892,  0.1715],
│                    [-0.0126,  0.8789,  1.1263]],
│           
│                   [[ 0.5674, -0.5892,  0.1715],
│                    [-0.0126,  0.8789,  1.1263]],
│           
│                   [[ 0.5674, -0.5892,  0.1715],
│                    [-0.0126,  0.8789,  1.1263]],
│           
│                   [[ 0.5674, -0.5892,  0.1715],
│                    [-0.0126,  0.8789,  1.1263]],
│           
│                   [[ 0.5674, -0.5892,  0.1715],
│                    [-0.0126,  0.8789,  1.1263]]])
└── 'x' --> <FastTreeValue 0x7fbdf2b39d60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 66.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbea0bdff40>
├── 'a' --> tensor([[ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263],
│                   [ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263],
│                   [ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263],
│                   [ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263],
│                   [ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263],
│                   [ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263],
│                   [ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263],
│                   [ 0.5674, -0.5892,  0.1715],
│                   [-0.0126,  0.8789,  1.1263]])
└── 'x' --> <FastTreeValue 0x7fbea0bdfe80>
    └── 'c' --> tensor([[-0.4409,  0.5967,  1.4386, -1.2170],
                        [-0.7715,  0.1039,  0.0344,  0.4698],
                 

In [23]:
%timeit t_cat(trees)

31.2 µs ± 76.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 92.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5674, -0.5892,  0.1715],
                [-0.0126,  0.8789,  1.1263]],
       
               [[ 0.5674, -0.5892,  0.1715],
                [-0.0126,  0.8789,  1.1263]],
       
               [[ 0.5674, -0.5892,  0.1715],
                [-0.0126,  0.8789,  1.1263]],
       
               [[ 0.5674, -0.5892,  0.1715],
                [-0.0126,  0.8789,  1.1263]],
       
               [[ 0.5674, -0.5892,  0.1715],
                [-0.0126,  0.8789,  1.1263]],
       
               [[ 0.5674, -0.5892,  0.1715],
                [-0.0126,  0.8789,  1.1263]],
       
               [[ 0.5674, -0.5892,  0.1715],
                [-0.0126,  0.8789,  1.1263]],
       
               [[ 0.5674, -0.5892,  0.1715],
                [-0.0126,  0.8789,  1.1263]]]),
    x: Batch(
           c: tensor([[[-0.4409,  0.5967,  1.4386, -1.2170],
                       [-0.7715,  0.1039,  0.0344,  0.4698],
                       [-1.0914,  1.5996,  1.2745, -1.0413]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 72.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263],
               [ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263],
               [ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263],
               [ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263],
               [ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263],
               [ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263],
               [ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263],
               [ 0.5674, -0.5892,  0.1715],
               [-0.0126,  0.8789,  1.1263]]),
    x: Batch(
           c: tensor([[-0.4409,  0.5967,  1.4386, -1.2170],
                      [-0.7715,  0.1039,  0.0344,  0.4698],
                      [-1.0914,  1.5996,  1.2745, -1.0413],
                      [-0.4409,  0.5967,  1.4386, -1.2170],
                      [-0.7715,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 314 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 558 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
